In [1]:
import os
import polars as pl
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import copy
import kaggle_evaluation.default_inference_server

In [2]:
# 1. Load Training Data
train_path = "/kaggle/input/hull-tactical-market-prediction/train.csv"
df_train = pl.read_csv(train_path)
print(df_train.head())

shape: (5, 98)
┌─────────┬─────┬─────┬─────┬───┬──────┬─────────────────┬────────────────┬────────────────────────┐
│ date_id ┆ D1  ┆ D2  ┆ D3  ┆ … ┆ V9   ┆ forward_returns ┆ risk_free_rate ┆ market_forward_excess_ │
│ ---     ┆ --- ┆ --- ┆ --- ┆   ┆ ---  ┆ ---             ┆ ---            ┆ returns                │
│ i64     ┆ i64 ┆ i64 ┆ i64 ┆   ┆ str  ┆ f64             ┆ f64            ┆ ---                    │
│         ┆     ┆     ┆     ┆   ┆      ┆                 ┆                ┆ f64                    │
╞═════════╪═════╪═════╪═════╪═══╪══════╪═════════════════╪════════════════╪════════════════════════╡
│ 0       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null ┆ -0.002421       ┆ 0.000301       ┆ -0.003038              │
│ 1       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null ┆ -0.008495       ┆ 0.000303       ┆ -0.009114              │
│ 2       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null ┆ -0.009624       ┆ 0.000301       ┆ -0.010243              │
│ 3       ┆ 0   ┆ 0   ┆ 0   ┆ … ┆ null ┆ 0.004662        ┆ 0.000299       ┆ 

In [3]:
pdf = df_train.to_pandas()

# price proxy
pdf["price_proxy"] = (1 + pdf["forward_returns"].shift(1).fillna(0)).cumprod()

# 1. price momentum
for w in [5, 21, 63]:
    pdf[f"ret_{w}d_mom"] = pdf["forward_returns"].shift(1).fillna(0).rolling(w).sum()

# 2. Moving averages
for w in [5, 21, 63]:
    ma_col = f"ma_{w}"
    pdf[ma_col] = pdf["price_proxy"].rolling(w).mean()
    pdf[f"price_over_ma_{w}"] = pdf["price_proxy"] / pdf[ma_col]

# 3. Rolling volatility
for w in [21, 63]:
    pdf[f"vol_{w}d"] = pdf["forward_returns"].shift(1).fillna(0).rolling(w).std()

# drawdown proxy (recent w days cum of returns)
for w in [21, 63]:
    roll_cum = pdf["forward_returns"].shift(1).fillna(0).rolling(w).sum()
    roll_max = roll_cum.rolling(w).max()
    pdf[f"dd_{w}d"] = roll_cum - roll_max

df_train = pl.from_pandas(pdf)

def generate_features(pdf):
    df = pdf.copy()
    
    # 1. Price Proxy
    df["price_proxy"] = (1 + df["returns"]).cumprod()

    # 2. Price Momentum
    for w in [5, 21, 63]:
        df[f"ret_{w}d_mom"] = df["returns"].rolling(w).sum()

    # 3. Moving Averages
    for w in [5, 21, 63]:
        ma_col = f"ma_{w}"
        df[ma_col] = df["price_proxy"].rolling(w).mean()
        df[f"price_over_ma_{w}"] = df["price_proxy"] / df[ma_col]

    # 4. Rolling Volatility
    for w in [21, 63]:
        df[f"vol_{w}d"] = df["returns"].rolling(w).std()

    # 5. Drawdown Proxy
    for w in [21, 63]:
        roll_cum = df["returns"].rolling(w).sum()
        roll_max = roll_cum.rolling(w).max()
        df[f"dd_{w}d"] = roll_cum - roll_max
        
    return df

In [4]:
# Load Training Data (Assuming df_train is already loaded)
pdf_train = df_train.to_pandas()
pdf_train = pdf_train.apply(pd.to_numeric, errors='coerce')

# Remove Future Data Leakage
# Since 'forward_returns' represents returns from T to T+1, it is unknown at time T.
# Therefore, applying shift(1) converts it to returns from T-1 to T (past data).
# Assign this to the 'returns' column to be used in the shared feature generation function.
pdf_train["returns"] = pdf_train["forward_returns"].shift(1).fillna(0)

# Generate features using the shared function
pdf_train = generate_features(pdf_train)

# Handle NaNs created by rolling windows
pdf_train = pdf_train.fillna(0)

# Convert back to Polars DataFrame (To maintain pipeline consistency)
df_train = pl.from_pandas(pdf_train)

In [5]:
# 2. Preprocessing
# Exclude non-feature columns
ignore_cols = ['date_id', 'forward_returns', 'market_forward_excess_returns', 'risk_free_rate', 'returns']
feature_cols = [c for c in df_train.columns if c not in ignore_cols]
target_col = 'market_forward_excess_returns'

# Convert to Pandas for CatBoost
X = df_train.select(feature_cols).to_pandas()
y = df_train.select(target_col).to_pandas().values.ravel()

# Convert any non-numeric objects to NaN.
X = X.apply(pd.to_numeric, errors='coerce')

# Fill NaN with mean value by using SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_imputed_array = imputer.fit_transform(X)
X = pd.DataFrame(X_imputed_array, columns=X.columns)

# Prepare data for volatility initialization (Last 60 days of returns)
train_returns = df_train.select('forward_returns').tail(60).to_numpy().flatten()

In [6]:
# 3. Time-Series Cross-Validation (Validation ONLY)
tscv = TimeSeriesSplit(n_splits=5) 
cv_scores = []

# Define models to compare in a dictionary
models_config = {
    "CatBoost": CatBoostRegressor(
        iterations=600, learning_rate=0.03, depth=6, 
        loss_function='RMSE', verbose=False, allow_writing_files=False, random_seed=42
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=600, learning_rate=0.03, max_depth=6, 
        verbosity=-1, random_state=42, n_jobs=-1
    ),
    "RandomForest": RandomForestRegressor(
        n_estimators=200, max_depth=10, 
        random_state=42, n_jobs=-1
    )
}

results = {}      # Store CV scores for each model
best_score = float('inf') # Initial value for finding minimum MSE
best_model_name = ""
best_model_instance = None

print("Starting Time-Series Cross-Validation (Comparing Models)...")
print("-" * 50)

# Loop through each model to perform CV
for model_name, model_instance in models_config.items():
    print(f"Testing Model: {model_name}")
    cv_scores = []
    
    for fold, (train_index, val_index) in enumerate(tscv.split(X)):
        X_train_cv, X_val_cv = X.iloc[train_index], X.iloc[val_index]
        y_train_cv, y_val_cv = y[train_index], y[val_index]
        
        # Initialize model
        if model_name == "CatBoost":
             # CatBoost requires specific parameter handling
             current_model = CatBoostRegressor(**model_instance.get_params())
        else:
             # Standard Scikit-Learn compatible models (RF, LGBM)
             current_model = model_instance.__class__(**model_instance.get_params())

        current_model.fit(X_train_cv, y_train_cv)
        
        preds = current_model.predict(X_val_cv)
        mse = mean_squared_error(y_val_cv, preds)
        cv_scores.append(mse)

        print(f"    Fold {fold+1} MSE: {mse:.6f}")
    
    # Calculate average MSE
    avg_mse = np.mean(cv_scores)
    results[model_name] = avg_mse
    print(f"  -> {model_name} Average CV MSE: {avg_mse:.6f}")
    
    # Update the best performing model (Lower MSE is better)
    if avg_mse < best_score:
        best_score = avg_mse
        best_model_name = model_name
        # Save the instance to retrain on full data later
        best_model_instance = model_instance

print("-" * 50)
print(f"🏆 Best Model: {best_model_name} (MSE: {best_score:.6f})")

Starting Time-Series Cross-Validation (Comparing Models)...
--------------------------------------------------
Testing Model: CatBoost
    Fold 1 MSE: 0.000151
    Fold 2 MSE: 0.000103
    Fold 3 MSE: 0.000188
    Fold 4 MSE: 0.000071
    Fold 5 MSE: 0.000133
  -> CatBoost Average CV MSE: 0.000129
Testing Model: LightGBM
    Fold 1 MSE: 0.000159
    Fold 2 MSE: 0.000140
    Fold 3 MSE: 0.000207
    Fold 4 MSE: 0.000082
    Fold 5 MSE: 0.000158
  -> LightGBM Average CV MSE: 0.000149
Testing Model: RandomForest
    Fold 1 MSE: 0.000156
    Fold 2 MSE: 0.000125
    Fold 3 MSE: 0.000186
    Fold 4 MSE: 0.000069
    Fold 5 MSE: 0.000135
  -> RandomForest Average CV MSE: 0.000134
--------------------------------------------------
🏆 Best Model: CatBoost (MSE: 0.000129)


In [7]:
# 4. Final Model Training
# Retrain the BEST model on the FULL dataset to capture the most recent market trends
print(f"Training Final Model ({best_model_name}) on ALL data...")

if best_model_name == "CatBoost":
    final_model = CatBoostRegressor(**best_model_instance.get_params())
else:
    final_model = best_model_instance.__class__(**best_model_instance.get_params())

final_model.fit(X, y)
print("Final model training completed.")

Training Final Model (CatBoost) on ALL data...
Final model training completed.


In [8]:
# 5. Volatility Management Class
class VolatilityController:
    def __init__(self, window_size=60, target_ratio=1.15, initial_returns=None):
        self.window_size = window_size
        self.target_ratio = target_ratio
        self.history = []
        
        # Initialize buffer with the end of training data
        if initial_returns is not None:
            for ret in initial_returns:
                self.history.append({'market_ret': ret, 'weight': 1.0})

    def calculate_safe_weight(self, raw_weight, current_lagged_return, last_weight):
        # Update history with realized return from the previous step
        if current_lagged_return is not None:
             self.history.append({'market_ret': current_lagged_return, 'weight': last_weight})
        
        # Maintain window size
        if len(self.history) > self.window_size:
            self.history.pop(0)
            
        # Warm-up check
        if len(self.history) < 10:
            return raw_weight
            
        # Calculate Volatility Ratio
        market_rets = np.array([x['market_ret'] for x in self.history])
        weights = np.array([x['weight'] for x in self.history])
        strategy_rets = weights * market_rets
        
        vol_bench = np.std(market_rets)
        vol_strat = np.std(strategy_rets)
        
        if vol_bench < 1e-7: 
            return raw_weight

        current_ratio = vol_strat / vol_bench
        
        # Apply scaling if ratio exceeds target (1.15 to be safe for 1.20 limit)
        scaling_factor = 1.0
        if current_ratio > self.target_ratio:
            scaling_factor = self.target_ratio / current_ratio
            
        return raw_weight * scaling_factor

In [9]:
# Initialize Controller
vol_manager = VolatilityController(window_size=60, target_ratio=1.15, initial_returns=train_returns)
history_returns = []
last_submitted_weight = 1.0 

# 6. Inference Function
def predict(test: pl.DataFrame) -> float:
    global last_submitted_weight, history_returns
    try:
        new_ret = test.select("lagged_forward_returns").item(0, 0)
        
        # 첫 번째 데이터가 null일 경우 처리
        if new_ret is None: 
            new_ret = 0.0
            
        history_returns.append(new_ret)
        
        # # 메모리 관리를 위해 필요한 최대 윈도우(63일) + 여유분(e.g., 100개)만 유지
        # if len(history_returns) > 100:
        #     history_returns.pop(0)
            
        # Real-time Feature Engineering
        hist_df = pd.DataFrame({"returns": history_returns})
        
        # 공통 함수 호출 (Step 3와 완벽히 동일한 로직 적용)
        hist_df_feat = generate_features(hist_df)
        
        # Extract the last row (Current Timestep T) and reset index for concatenation
        current_engineered_feat = hist_df_feat.iloc[[-1]].copy().reset_index(drop=True)
        
        # Combine Raw Features & Engineered Features
        # Convert raw test data (M*, P*, etc.) to Pandas
        test_pd = test.to_pandas()
        
        # Concatenate Raw Features + Engineered Features
        X_test_full = pd.concat([test_pd, current_engineered_feat], axis=1)
        X_test = X_test_full.reindex(columns=feature_cols, fill_value=0)
        
        # Prediction using the FINAL MODEL (Trained on all data)
        pred_excess_return = final_model.predict(X_test)[0]
        
        # Sigmoid Betting Strategy (Output 0 ~ 2)
        # Using a fixed scale factor (heuristic)
        scale_factor = 10.0
        if pred_excess_return < 0:
            # Negative: Aggressive scaling
            # Even a small negative prediction will drop weight significantly
            scale_factor *= 5  # 5x more sensitive to downside
        
        sigmoid_value = 1 / (1 + np.exp(-pred_excess_return * scale_factor))
        raw_weight = 2.0 * sigmoid_value
        
        # Volatility Control
        final_weight = vol_manager.calculate_safe_weight(raw_weight, new_ret, last_submitted_weight)
        
        # Hard clip to ensure valid submission
        final_weight = max(0.0, min(2.0, final_weight))
        
        # Update state for next iteration
        last_submitted_weight = final_weight

        # Print test information
        print(f"current test : {test.select('date_id').item(0, 0)}")
        print(f"pred_excess_return : {pred_excess_return}")
        print(f"raw_weight : {raw_weight}")
        print(f"final_weight : {final_weight}")
        print()
        
        return float(final_weight)
        
    except Exception as e:
        print(e)
        # Safety fallback
        return 0.0

# Start Inference Server
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))

current test : 8980
pred_excess_return : 0.010577032719402874
raw_weight : 1.05283591490294
final_weight : 1.05283591490294

current test : 8981
pred_excess_return : 0.007735800350770799
raw_weight : 1.0386597245205733
final_weight : 1.0386597245205733

current test : 8982
pred_excess_return : 0.012769460007115332
raw_weight : 1.0637606839029314
final_weight : 1.0637606839029314

current test : 8983
pred_excess_return : 0.013328664322287232
raw_weight : 1.0665448348600668
final_weight : 1.0665448348600668

current test : 8984
pred_excess_return : 0.009621347743269334
raw_weight : 1.0480696625642065
final_weight : 1.0480696625642065

current test : 8985
pred_excess_return : 0.009443515561994438
raw_weight : 1.0471825185510155
final_weight : 1.0471825185510155

current test : 8986
pred_excess_return : 0.00943034887316593
raw_weight : 1.04711683146069
final_weight : 1.04711683146069

current test : 8987
pred_excess_return : 0.008516439137097726
raw_weight : 1.0425564770440459
final_weight